In [1359]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from pandas import read_html

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [1360]:
#Constants for accessing DataFrame data pulled from web
QB = 0
RB = 1
WR = 2
TE = 3
DST = 4
K = 5

DATA = 0
KEYS = 1

In [1361]:
#Set to create unique in house DataFrame 

tupOffense = [('', 'Game'),        
              ('', 'Pts'),
              ('Passing', 'Att'),
              ('Passing', 'Cmp'),
              ('Passing', 'Yds'),
              ('Passing', 'TD'),
              ('Passing', 'Int'),
              ('Passing', '2Pt'),
              ('Rushing', 'Att'),
              ('Rushing', 'Yds'),
              ('Rushing', 'TD'),
              ('Rushing', '2Pt'),
              ('Receiving', 'Rec'),
              ('Receiving', 'Yds'),
              ('Receiving', 'TD'),
              ('Receiving', '2Pt'),
              ('Fumbles', 'FL'),
              ('Fumbles', 'TD')
             ]   

tupDef = [('', 'Opp'),
          ('', 'Pts*'),
          ('Def', 'Sack'), 
          ('Def', 'Int'),
          ('Def', 'Saf'), 
          ('Def', 'FR'), 
          ('Def', 'Blk'), 
          ('Def', 'TD'), 
          ('Def', 'PA'),
          ('Def', 'PassYds/G'), 
          ('Def', 'RushYds/G'), 
          ('Def', 'TotYds/G')
         ]

tupKicker = [('', 'Game'), 
             ('', 'Pts*'),          
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

#Set to create unique in house DataFrame of master Dataframe that will have all players and seasons
masterTup = [('', 'PPR'),
             ('', 'Pos'),
             ('', 'Team'),
             ('', 'Bye'),
             ('', 'Pts'),
             ('', 'Week'),
             ('', 'Year'),
             ('', 'Game'),
             ('', 'Opp'),
             ('Passing', 'Att'),
             ('Passing', 'Cmp'),
             ('Passing', 'Yds'),
             ('Passing', 'TD'),
             ('Passing', 'Int'),
             ('Passing', '2Pt'),
             ('Rushing', 'Att'),
             ('Rushing', 'Yds'),
             ('Rushing', 'TD'),
             ('Rushing', '2Pt'),
             ('Receiving', 'Rec'),
             ('Receiving', 'Yds'),
             ('Receiving', 'TD'),
             ('Receiving', '2Pt'),
             ('Fumbles', 'FL'),
             ('Fumbles', 'TD'),
             ('Defense', 'Sack'),
             ('Defense', 'Int'),
             ('Defense', 'Saf'),
             ('Defense', 'FR'), 
             ('Defense', 'Blk'),
             ('Defense', 'TD'),
             ('Defense', 'PA'),
             ('Defense', 'PassYds/G'),
             ('Defense', 'RushYds/G'),
             ('Defense', 'TotYds/G'),
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

In [1362]:
#Set to change the index names 

indexDEF = ['Bye', 'Pts*', 'Sack', 'Int', 'Saf', 'FR', 'Blk', 'TD', 'PA',
       'PassYds/G', 'RushYds/G', 'TotYds/G']

indexKicker = ['Bye', 'Pts*', 'XPA', 'XPM', 'FGA', 'FGM', '50+']


In [1363]:
#Load data from web for 2017 season
def loadWebData(frameArray):
    url_qb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2017&wk=all&rules=1'
    url_rb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr=2017&wk=all&rules=1'
    url_wr_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
    url_te_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=TE&yr=2017&wk=all&rules=1'
    url_def_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr=2017&wk=all&rules=1'
    url_kicker_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=K&yr=2017&wk=all&rules=1'
    
    #Array [0] is where the data is stored. [1] stores the rules 
    frameArray.insert(QB, pd.io.html.read_html(url_qb_season17)[0])
    frameArray.insert(RB, pd.io.html.read_html(url_rb_season17)[0])
    frameArray.insert(WR, pd.io.html.read_html(url_wr_season17)[0])
    frameArray.insert(TE, pd.io.html.read_html(url_te_season17)[0])
    frameArray.insert(DST, pd.io.html.read_html(url_def_season17)[0]) 
    frameArray.insert(K, pd.io.html.read_html(url_kicker_season17)[0]) 

In [1364]:
#Fix name that contains the next line string '\xa0'
#nameIndex: Empty index that will contain the fixed name
#data: Pandas Series type that contains the names of each player
def nameFix(nameIndex, data):
    #Get name from Player column and save to nameIndex
    NAME_FIX_OFFSET = 3
    index = nameIndex

    for i in data:
        index.append(i)


    size = len(index)

    for i in range(size):
        a1 = index[i]
        count = 0

        for x in a1:
            count += 1
            if x == u'\xa0':
                break

        count -= NAME_FIX_OFFSET
        index[i] = a1[:count]
    
    return index

In [1365]:
def renameIndex(dframe, teamName):

    d1 = dict()
    index = 0

    for i in teamName:
        d1[index] = i
        index += 1
        
    dframe = (dframe.rename(d1, axis = 'index'))
    return(dframe)
    

In [1366]:
def trimName(teamName):    
    arryIndex = len(teamName)
    
    
    for a1 in range(arryIndex):
        #a1 is index for the string array teamName 
        strLen = len(teamName.iloc[a1])
        for b1 in teamName.iloc[a1][::-1]:
            if b1.isupper():
                strLen -= 1
            else:
                break
                
        teamName.iat[a1] = (teamName.iloc[a1][:strLen])
       
    return teamName

In [1367]:
def setDataFrame(dframe, dataType, index, fixXa0):
    nameIndex = []
    dataMatrix = []
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    if(fixXa0):
        nameIndex = nameFix(nameIndex, frameArray[dataType]['\xa0'].get("Player")) 
        dframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    else:
        dframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(dataType == DST):
            nameIndex = frameArray[dataType].get('Team')
            nameIndex = trimName(nameIndex)
        elif(dataType == K):
            nameIndex = nameFix(nameIndex, frameArray[dataType].get('Player'))               
                               
        d1 = dict()
        index = 0

        for i in nameIndex:
            d1[index] = i
            index += 1
        
        dframe = (dframe.rename(d1, axis = 'index'))
        dframe = dframe.rename({'Pts*': 'Pts'}, axis='columns')
        
        
        
    return(dframe)



In [1368]:
def createIndex(keyList):
    position = keyList['Pos'] 
    index = []
    
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        index = pd.MultiIndex.from_tuples(tupOffense)
    elif(position == 'K'):
        index = pd.MultiIndex.from_tuples(tupKicker)
    elif(position == 'DST'):
        index = pd.MultiIndex.from_tuples(tupDef)
        
    return index

In [1369]:
def pullDataWeb():
    ffb_year = [i for i in range(2010, 2019)]
    ffb_stdscore = [1,0]
    ffb_week = [i for i in range(1,18)]
    ffb_pos = ['QB', 'RB', 'WR', 'TE', 'K', 'DST']
    frameArray = []

    ffb_d1 = {'StdScore': ffb_stdscore, 'Year':ffb_year, 'Week': ffb_week, 'Pos': ffb_pos}

    keyList = []
    count = 0
    url = []


    #Create url string for every week. Save scoring, year, week, and pos for future reference
    for scoring in ffb_d1['StdScore']:
        for year in ffb_d1['year']:
            for week in ffb_d1['week']:
                for pos in ffb_d1['pos']:
                    url.append("https://www.footballdb.com/fantasy-football/index.html?pos={p}&yr={y}&wk={w}&rules={r}"\
                    .format(p=pos, y=year, w=week, r=scoring))
                    keylist.append({'stdscore':scoring, 'year':year, 'week':week, 'pos':pos})
                    
    #frame array will be a list of tuples. first value of tuple is dframe. second is
    #dictionary that contains the keys
    for i in url:
        framearray.append((pd.io.html.read_html(i)[0], keylist[count]))
        count += 1 

In [1370]:
def setDataFrame_weeks(dframe, keyList, index):
    nameIndex = []  #This is the name of the team or player
    dataMatrix = [] #This is the data going into dataframe
    position = keyList['Pos'] #Get the position of the player
    dkeylist = {
            ('','Pos'): keyList['Pos'], 
            ('', 'PPR'): ('No' if keyList['StdScore'] else 'Yes'),
            ('', 'Week'): keyList['Week'],
            ('', 'Year'): keyList['Year']
         }
    
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    #Fix the names for QB, WR, TE, RB
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        nameIndex = nameFix(nameIndex, dframe['\xa0'].get("Player")) 
        retframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    #Fix the names for Kicker and Defense
    elif(position=='DST' or position=='K'):
        dname = dict()
        count = 0
        retframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(position == 'DST'):
            nameIndex = dframe.get('Team')
            nameIndex = trimName(nameIndex)
        elif(position == 'K'):
            nameIndex = nameFix(nameIndex, dframe.get('Player'))               
                               
        for i in nameIndex:
            dname[count] = i
            count += 1
        #Rename the frame and change the Pts* to Pts
        retframe = (retframe.rename(dname, axis = 'index'))
        retframe = retframe.rename({'Pts*': 'Pts'}, axis='columns')
        
    #Create the dataframe for the keys using keyList argument and concat
    #with the returning frame
    kframe = DataFrame(dkeylist, nameIndex)
    retframe = pd.concat([retframe, kframe], axis=1, join_axes=[retframe.index])    
        
    return(retframe)


In [1508]:
def setDataFrame_test(dframe, keyList, index):
    nameIndex = []  #This is the name of the team or player
    dataMatrix = [] #This is the data going into dataframe
    position = keyList['Pos'] #Get the positionof the data
    retframe = dframe
    
    dkeylist = {
            ('','Pos'): keyList['Pos'], 
            ('', 'PPR'): ('No' if keyList['StdScore'] else 'Yes'),
            ('', 'Week'): keyList['Week'],
            ('', 'Year'): keyList['Year']
         }
    

    
    #For QB, WR, TE, and RB, rename the columns and index to be more legible
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        
        d1 = dict(zip(dframe.columns.levels[0], 
            ['Passing', 'Passing','Passing', 'Passing', 'Rushing',\
             'Rushing', 'Rushing', 'Receiving',\
             'Receiving', 'Receiving', 'Receiving',\
             'Fumbles', 'Fumbles', 'Passing',\
             'Passing', 'Rushing', '']))
        
        nameIndex = nameFix(nameIndex, dframe['\xa0'].get("Player")) 
        #retframe = dframe.rename(columns=d1, level=0)
        
        d2 = dict(zip(retframe[('\xa0','Player')], nameIndex))
        retframe = retframe.set_index(('\xa0','Player'))
        retframe = retframe.rename_axis('')
        retframe = retframe.rename(d2, axis='index')
        retframe.columns = index
        retframe = retframe.rename({'Pts*': 'Pts'}, axis='columns')

    #Fix the names for Kicker and Defense
    elif(position=='DST' or position=='K'):
        dname = dict()
        count = 0
        #retframe = DataFrame(dataMatrix, columns = index)
        #retframe = dframe
        #retframe.columns = index
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(position == 'DST'):
            nameIndex = dframe.get('Team')
            nameIndex = trimName(nameIndex)
            
            d2 = dict(zip(retframe[('Team')], nameIndex))
            retframe = retframe.set_index(('Team'))
            retframe = retframe.rename_axis('')
            retframe = retframe.rename(d2, axis='index')
            retframe.columns = index
            
        elif(position == 'K'):
            nameIndex = nameFix(nameIndex, dframe.get('Player'))   
            d2 = dict(zip(retframe[('Player')], nameIndex))
            retframe = retframe.set_index(('Player'))
            retframe = retframe.rename_axis('')
            retframe = retframe.rename(d2, axis='index')
            retframe.columns = index
            #return(retframe)
                               
        #for i in nameIndex:
        #   dname[count] = i
        #   count += 1
        #Rename the frame and change the Pts* to Pts
        #retframe = (retframe.rename(dname, axis = 'index'))
        retframe = retframe.rename({'Pts*': 'Pts'}, axis='columns')
        
    #Create the dataframe for the keys using keyList argument and concat
    #with the returning frame
    kframe = DataFrame(dkeylist, nameIndex)
    retframe = pd.concat([retframe, kframe], axis=1, join_axes=[retframe.index])    
        
    return(retframe)


In [1517]:
#MAIN STARTS HERE
frameArray = []

In [1518]:
columns = pd.MultiIndex.from_tuples(masterTup)

In [1525]:
frameMaster = DataFrame(columns = columns)
test_count = 0

for tup in tframeArray:
    index = createIndex(tup[KEYS])
    dframe = setDataFrame_test(tup[DATA], tup[KEYS], index)
    frameArray.append(dframe)
    frameMaster = pd.concat([frameMaster, dframe])
    
    test_count += 0
    if test_count == 6:
        break

In [1528]:
frameMaster.loc['Tom Brady']

Def ... Passing  \
           Bye    Game  Opp  PPR Pos   Pts Team Week  Year Blk ...      TD   
Tom Brady  NaN  CIN@NE  NaN   No  QB  28.0  NaN    1  2010 NaN ...       3   
Tom Brady  NaN  NE@NYJ  NaN   No  QB  16.0  NaN    2  2010 NaN ...       2   
Tom Brady  NaN  BUF@NE  NaN   No  QB  28.0  NaN    3  2010 NaN ...       3   
Tom Brady  NaN  NE@MIA  NaN   No  QB  12.0  NaN    4  2010 NaN ...       1   
Tom Brady  NaN  BAL@NE  NaN   No  QB  13.0  NaN    6  2010 NaN ...       1   
Tom Brady  NaN   NE@SD  NaN   No  QB  12.0  NaN    7  2010 NaN ...       1   
Tom Brady  NaN  MIN@NE  NaN   No  QB  15.0  NaN    8  2010 NaN ...       1   
Tom Brady  NaN  NE@CLE  NaN   No  QB  20.0  NaN    9  2010 NaN ...       2   
Tom Brady  NaN  NE@PIT  NaN   No  QB  38.0  NaN   10  2010 NaN ...       3   
Tom Brady  NaN  IND@NE  NaN   No  QB  19.0  NaN   11  2010 NaN ...       2   
Tom Brady  NaN  NE@DET  NaN   No  QB  37.0  NaN   12  2010 NaN ...       4   
Tom Brady  NaN  NYJ@NE  NaN   No  QB  37.0  NaN   13  2010 NaN ...       4   
Tom Brady  NaN  NE@CHI  NaN   No  QB  26.0  NaN   14  2010 NaN ...       2   
Tom Brady  NaN   GB@NE  NaN   No  QB  18.0  NaN   15  2010 NaN ...       2   
Tom Brady  NaN  NE@BUF  NaN   No  QB  24.0  NaN   16  2010 NaN ...       3   
Tom Brady  NaN  MIA@NE  NaN   No  QB  19.0  NaN   17  2010 NaN ...       2   
Tom Brady  NaN  NE@MIA  NaN   No  QB  42.0  NaN    1  2011 NaN ...       4   
Tom Brady  NaN   SD@NE  NaN   No  QB  34.0  NaN    2  2011 NaN ...       3   
Tom Brady  NaN  NE@BUF  NaN   No  QB  31.0  NaN    3  2011 NaN ...       4   
Tom Brady  NaN  NE@OAK  NaN   No  QB  21.0  NaN    4  2011 NaN ...       2   
Tom Brady  NaN  NYJ@NE  NaN   No  QB  16.0  NaN    5  2011 NaN ...       1   
Tom Brady  NaN  DAL@NE  NaN   No  QB  20.0  NaN    6  2011 NaN ...       2   
Tom Brady  NaN  NE@PIT  NaN   No  QB  19.0  NaN    8  2011 NaN ...       2   
Tom Brady  NaN  NYG@NE  NaN   No  QB  20.0  NaN    9  2011 NaN ...       2   
Tom Brady  NaN  NE@NYJ  NaN   No  QB  31.0  NaN   10  2011 NaN ...       3   
Tom Brady  NaN   KC@NE  NaN   No  QB  21.0  NaN   11  2011 NaN ...       2   
Tom Brady  NaN  NE@PHI  NaN   No  QB  34.0  NaN   12  2011 NaN ...       3   
Tom Brady  NaN  IND@NE  NaN   No  QB  23.0  NaN   13  2011 NaN ...       2   
Tom Brady  NaN  NE@WAS  NaN   No  QB  30.0  NaN   14  2011 NaN ...       3   
Tom Brady  NaN  NE@DEN  NaN   No  QB  30.0  NaN   15  2011 NaN ...       2   
...        ...     ...  ...  ...  ..   ...  ...  ...   ...  .. ...     ...   
Tom Brady  NaN  NE@NYJ  NaN  Yes  QB  13.0  NaN   16  2015 NaN ...       1   
Tom Brady  NaN  NE@MIA  NaN  Yes  QB   5.0  NaN   17  2015 NaN ...       0   
Tom Brady  NaN  NE@CLE  NaN  Yes  QB  35.0  NaN    5  2016 NaN ...       3   
Tom Brady  NaN  CIN@NE  NaN  Yes  QB  33.0  NaN    6  2016 NaN ...       3   
Tom Brady  NaN  NE@PIT  NaN  Yes  QB  21.0  NaN    7  2016 NaN ...       2   
Tom Brady  NaN  NE@BUF  NaN  Yes  QB  37.0  NaN    8  2016 NaN ...       4   
Tom Brady  NaN  SEA@NE  NaN  Yes  QB  10.0  NaN   10  2016 NaN ...       0   
Tom Brady  NaN   NE@SF  NaN  Yes  QB  36.0  NaN   11  2016 NaN ...       4   
Tom Brady  NaN  NE@NYJ  NaN  Yes  QB  23.0  NaN   12  2016 NaN ...       2   
Tom Brady  NaN   LA@NE  NaN  Yes  QB  16.0  NaN   13  2016 NaN ...       1   
Tom Brady  NaN  BAL@NE  NaN  Yes  QB  32.0  NaN   14  2016 NaN ...       3   
Tom Brady  NaN  NE@DEN  NaN  Yes  QB   7.0  NaN   15  2016 NaN ...       0   
Tom Brady  NaN  NYJ@NE  NaN  Yes  QB  26.0  NaN   16  2016 NaN ...       3   
Tom Brady  NaN  NE@MIA  NaN  Yes  QB  31.0  NaN   17  2016 NaN ...       3   
Tom Brady  NaN   KC@NE  NaN  Yes  QB  10.0  NaN    1  2017 NaN ...       0   
Tom Brady  NaN   NE@NO  NaN  Yes  QB  35.0  NaN    2  2017 NaN ...       3   
Tom Brady  NaN  HOU@NE  NaN  Yes  QB  46.0  NaN    3  2017 NaN ...       5   
Tom Brady  NaN  CAR@NE  NaN  Yes  QB  24.0  NaN    4  2017 NaN ...       2   
Tom Brady  NaN   NE@TB  NaN  Yes  QB  15.0  NaN    5  2017 NaN ...       1   
Tom

In [1320]:
dframe[('','Player')]


        d1 = dict(zip(dframe.columns.levels[0], 
            ['Passing', 'Passing','Passing', 'Passing', 'Rushing',\
             'Rushing', 'Rushing', 'Receiving',\
             'Receiving', 'Receiving', 'Receiving',\
             'Fumbles', 'Fumbles', 'Passing',\
             'Passing', 'Rushing', '']))

0                               hi
1                Tom BradyT. Brady
2          David GarrardD. Garrard
3              Jay CutlerJ. Cutler
4              Michael VickM. Vick
5           Philip RiversP. Rivers
6           Carson PalmerC. Palmer
7            Eli ManningE. Manning
8     Matt HasselbeckM. Hasselbeck
9              Vince YoungV. Young
10          Josh FreemanJ. Freeman
11                Tony RomoT. Romo
12              Kyle OrtonK. Orton
13       Derek AndersonD. Anderson
14         Aaron RodgersA. Rodgers
15              Drew BreesD. Brees
16       Jason CampbellJ. Campbell
17         Trent EdwardsT. Edwards
18        Jake DelhommeJ. Delhomme
19             Brett FavreB. Favre
20         Sam BradfordS. Bradford
21            Matt SchaubM. Schaub
22                Matt RyanM. Ryan
23            Matt CasselM. Cassel
24              Matt MooreM. Moore
25         Donovan McNabbD. McNabb
26              Chad HenneC. Henne
27            Dennis DixonD. Dixon
28             Joe F

# TEST SECTION STARTS AFTER THIS LINE 

Try to extract 2010 QBs only

In [1529]:
frameMaster.filter(items = [('','Game'), ('', 'Bye')])
PPR = frameMaster[('','PPR')] == 'Yes'

In [1530]:
Year = frameMaster[('', 'Year')] == 2015

In [1543]:
Pos_QB = frameMaster[('', 'Pos')] == 'QB'
Week = frameMaster['', 'Week'] < 18
Pts = frameMaster['', 'Pts'] > 50

In [1544]:
df = frameMaster[PPR & Pts]

In [1545]:
df

Def ...   \
                    Bye     Game  Opp  PPR Pos   Pts Team Week  Year Blk ...    
Michael Vick        NaN  PHI@WAS  NaN  Yes  QB  57.0  NaN   10  2010 NaN ...    
Aaron Rodgers       NaN   DEN@GB  NaN  Yes  QB  53.0  NaN    4  2011 NaN ...    
Matt Flynn          NaN   DET@GB  NaN  Yes  QB  52.0  NaN   17  2011 NaN ...    
Doug Martin         NaN   TB@OAK  NaN  Yes  RB  51.0  NaN    9  2012 NaN ...    
Peyton Manning      NaN  BAL@DEN  NaN  Yes  QB  60.0  NaN    1  2013 NaN ...    
Nick Foles          NaN  PHI@OAK  NaN  Yes  QB  59.0  NaN    9  2013 NaN ...    
Jamaal Charles      NaN   KC@OAK  NaN  Yes  RB  51.0  NaN   15  2013 NaN ...    
Ben Roethlisberger  NaN  IND@PIT  NaN  Yes  QB  56.0  NaN    8  2014 NaN ...    
Ryan Fitzpatrick    NaN  TEN@HOU  NaN  Yes  QB  51.0  NaN   13  2014 NaN ...    
Drew Brees          NaN   NYG@NO  NaN  Yes  QB  58.0  NaN    8  2015 NaN ...    
Cam Newton          NaN  CAR@NYG  NaN  Yes  QB  52.0  NaN   15  2015 NaN ...    

                   Passing      Receiving             Rushing              
                        TD  Yds       2Pt Rec TD  Yds     2Pt Att TD  Yds  
Michael Vick             4  333         0   0  0    0       0   8  2   80  
Aaron Rodgers            4  408         0   0  0    0       0   9  2   36  
Matt Flynn               6  480         0   0  0    0       0   2  0    0  
Doug Martin              0    0         0   4  0   21       0  25  4  251  
Peyton Manning           7  462         0   0  0    0       0   2  0   -2  
Nick Foles               7  406         0   0  0    0       0   4  0   10  
Jamaal Charles           0    0         0   8  4  195       0   8  1   20  
Ben Roethlisberger       6  522         0   0  0    0       0   1  0   -1  
Ryan Fitzpatrick         6  358         0   0  0    0       0   2  0   10  
Drew Brees               7  505         0   0  0    0       0   1  0    1  
Cam Newton               5  340         0   0  0    0       0   8  0  100  

[11 rows x 50 columns]

In [1117]:
df['Tom Brady']


KeyError: 'Tom Brady'

In [1047]:
week

Tom Brady              1
Marcus Mariota         1
Carson Palmer          1
Alex Smith             1
Tony Romo              1
Aaron Rodgers          1
Philip Rivers          1
Nick Foles             1
Andy Dalton            1
Andrew Luck            1
Ben Roethlisberger     1
Matt Ryan              1
Drew Brees             1
Jameis Winston         1
Russell Wilson         1
Tyrod Taylor           1
Matthew Stafford       1
Ryan Fitzpatrick       1
Jay Cutler             1
Sam Bradford           1
Ryan Tannehill         1
Cam Newton             1
Matt McGloin           1
Johnny Manziel         1
Brian Hoyer            1
Ryan Mallett           1
Blake Bortles          1
Colin Kaepernick       1
Kirk Cousins           1
Teddy Bridgewater      1
                      ..
Philip Rivers         17
Tyrod Taylor          17
A.J. McCarron         17
Blaine Gabbert        17
Drew Brees            17
Jameis Winston        17
Jay Cutler            17
Aaron Rodgers         17
Ryan Fitzpatrick      17


In [1048]:
d1 = {'Yds':yds, 'Week':week}

In [1062]:
d1 = DataFrame(d1)
d2 = d1.loc['Tom Brady']

d2.iloc[1][1]

'466'

In [1030]:
#sns.lmplot('Pts', 'Week', data=d2, fit_reg = True)